In [1]:
import numpy as np
import pandas as pd
import os
from tqdm import tqdm

In [64]:
if 'ygong' in os.getcwd():
    filepath = "../data"
    dir_out = "../output"
else:
    filepath = "/home/gong/Documents/Kaggle_July2019"
    dir_out = "/home/gong/Documents/Kaggle_July2019/output"

def load_data(filepath):
    train = pd.read_csv(os.path.join(filepath, 'train.csv'))
    test = pd.read_csv(os.path.join(filepath, 'test.csv'))
    submit = pd.read_csv(os.path.join(filepath, 'sample_submission.csv'))
    structures = pd.read_csv(os.path.join(filepath, 'structures.csv'))

    print('Train dataset shape is -> rows: {} cols:{}'.format(train.shape[0], train.shape[1]))
    print('Test dataset shape is  -> rows: {} cols:{}'.format(test.shape[0], test.shape[1]))
    print('Sample submission dataset shape is  -> rows: {} cols:{}'.format(submit.shape[0], submit.shape[1]))
    print('Structures dataset shape is  -> rows: {} cols:{}'.format(structures.shape[0], structures.shape[1]))
    print('\n')

    return train, test, submit, structures

train, test, submit, structures = load_data(filepath)

Train dataset shape is -> rows: 4658147 cols:6
Test dataset shape is  -> rows: 2505542 cols:5
Sample submission dataset shape is  -> rows: 2505542 cols:2
Structures dataset shape is  -> rows: 2358657 cols:6




In [67]:
structures.head(5)
# structures["atom"].unique()

,molecule_name,atom_index,atom,x,y,z
0,dsgdb9nsd_000001,0,C,-0.012698,1.085804,0.008001
1,dsgdb9nsd_000001,1,H,0.002150,-0.006031,0.001976
2,dsgdb9nsd_000001,2,H,1.011731,1.463751,0.000277
3,dsgdb9nsd_000001,3,H,-0.540815,1.447527,-0.876644
4,dsgdb9nsd_000001,4,H,-0.523814,1.437933,0.906397


In [54]:
# split number of interventing bonds and bond combination
def extract_n_bonds(row):
    return int(row['type'][0])
def extract_type_bond(row):
    return row['type'][2:]

train['n_bonds'] = train.agg(extract_n_bonds, axis=1)
train['type_bond'] = train.agg(extract_type_bond, axis=1)
test['n_bonds'] = test.agg(extract_n_bonds, axis=1)
test['type_bond'] = test.agg(extract_type_bond, axis=1)

In [63]:
# covlent radii
dict_radii = {'HC':107, 'HH':63, 'HN':102.5} 
dict_radii

{'HC': 107, 'HH': 63, 'HN': 102.5}

In [81]:
def get_bond_length(df, structures):
    for i in range(len(df)):
        a= train[['molecule_name', 'atom_index_0', 'atom_index_1']].values[i]
        a[]
a
structures.pivot_table(index=['molecule_name', 'atom_index']).loc['dsgdb9nsd_000002', 0].values

numpy.ndarray

In [96]:
a = train[['molecule_name', 'atom_index_0', 'atom_index_1']].values[0]
a

array(['dsgdb9nsd_000001', 1, 0], dtype=object)

In [97]:
structures.pivot_table(index=['molecule_name', 'atom_index']).loc[a[0], a[2]]

x   -0.012698
y    1.085804
z    0.008001
Name: (dsgdb9nsd_000001, 0), dtype: float64

In [98]:
test.head()

,id,molecule_name,atom_index_0,atom_index_1,type
0,4658147,dsgdb9nsd_000004,2,0,2JHC
1,4658148,dsgdb9nsd_000004,2,1,1JHC
2,4658149,dsgdb9nsd_000004,2,3,3JHH
3,4658150,dsgdb9nsd_000004,3,0,1JHC
4,4658151,dsgdb9nsd_000004,3,1,2JHC


In [79]:
import numpy as np
p1 = np.array([x1_coords, y1_coords, z1_coords])
p2 = np.array([x2_coords, y2_coords, z2_coords])

squared_dist = np.sum((p1-p2)**2, axis=0)
dist = np.sqrt(squared_dist)

AttributeError: module 'numpy' has no attribute 'dist'

In [ ]:
train.to_csv(os.path.join(dir_out, "_train.csv"), index=False)
print("Saved training dataset to {}".format(os.path.join(dir_out, "_train.csv")))

test.to_csv(os.path.join(dir_out, "_test.csv"), index=False)
print("Saved test dataset to {}".format(os.path.join(dir_out, "_test.csv")))